<a href="https://colab.research.google.com/github/pranao0609/DL_Projects/blob/main/8_Contrastive_SimCLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import random

In [2]:
simclr_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [3]:
class SimCLRDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        return simclr_transform(img), simclr_transform(img)

    def __len__(self):
        return len(self.dataset)

In [4]:
base_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
train_dataset = SimCLRDataset(base_dataset)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

100%|██████████| 170M/170M [00:04<00:00, 41.9MB/s]


In [5]:
class SimCLR(nn.Module):
    def __init__(self):
        super(SimCLR, self).__init__()
        base = models.resnet18(pretrained=False)
        self.encoder = nn.Sequential(*list(base.children())[:-1])
        self.projector = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, x):
        x = self.encoder(x).squeeze()
        x = self.projector(x)
        return F.normalize(x, dim=1)

In [6]:
def nt_xent_loss(z1, z2, temperature=0.5):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)
    sim = sim / temperature

    labels = torch.arange(N).repeat(2)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    labels = labels.to(z.device)

    mask = torch.eye(2*N, dtype=torch.bool).to(z.device)
    sim.masked_fill_(mask, -9e15)

    sim_exp = torch.exp(sim)
    sim_sum = sim_exp.sum(dim=1, keepdim=True)
    log_prob = sim - torch.log(sim_sum)
    loss = - (labels * log_prob).sum(dim=1) / labels.sum(dim=1)
    return loss.mean()

In [7]:
model = SimCLR()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
for epoch in range(1):
    for x1, x2 in train_loader:
        z1 = model(x1)
        z2 = model(x2)
        loss = nt_xent_loss(z1, z2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Contrastive Loss: {loss.item():.4f}")

Epoch 1, Contrastive Loss: 4499999907905536.0000
